## Preparando os Preditores

In [ ]:
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [ ]:
wine_quality = fetch_ucirepo(id=186)

X = wine_quality.data.features # Preditores
Y = wine_quality.data.targets # Output

# metadata
print(wine_quality.metadata)

print(wine_quality.variables)

{'uci_id': 186, 'name': 'Wine Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/186/wine+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/186/data.csv', 'abstract': 'Two datasets are included, related to red and white vinho verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], http://www3.dsi.uminho.pt/pcortez/wine/).', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 4898, 'num_features': 11, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['quality'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Wed Nov 15 2023', 'dataset_doi': '10.24432/C56S3T', 'creators': ['Paulo Cortez', 'A. Cerdeira', 'F. Almeida', 'T. Matos', 'J. Reis'], 'intro_paper': {'ID': 252, 'type': 'NATIVE', 'title': 'Modeling wine preferences

In [ ]:
df_white = pd.read_csv("../dataset/winequality-white.csv", sep=';',index_col=False)
df_white['type'] = 1
df_red = pd.read_csv("../dataset/winequality-red.csv", sep=';',index_col=False)
df_wine = pd.concat([df_red, df_white], ignore_index=True)
df_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,NaN
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,NaN
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,NaN
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,NaN
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,1.0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,1.0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,1.0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,1.0


In [ ]:
X = df_wine.drop("quality", axis=1)
X = X.drop("type", axis=1)
y = df_wine["quality"]

In [ ]:
# shuffle indexes
idx = np.arange(len(df_wine))
np.random.seed(42)
np.random.shuffle(idx)

# 80% train, 20% test
split_point = int(0.8 * len(df_wine))

train_idx = idx[:split_point]
test_idx  = idx[split_point:]

X_train = X.iloc[train_idx]
y_train = y.iloc[train_idx]

X_test = X.iloc[test_idx]
y_test = y.iloc[test_idx]

In [ ]:
X_train_scaled = (X_train - X_train.mean()) / X_train.std()
X_test_scaled = (X_test - X_train.mean()) / X_train.std()

##Learning a L2 or L1 penalised linear regression

### Ridge Function

#### Ridge Implementado

In [ ]:
def custom_ridge_regression(X, y, lamb):
  n_features = X.shape[1]
  I = np.eye(n_features)
  return np.linalg.inv(X.T @ X + lamb * I) @ (X.T @ y)


Adicionando o Intercepto:

In [ ]:
X_train_adj = np.hstack([np.ones((X_train_scaled.shape[0], 1)), X_train_scaled])
X_test_adj  = np.hstack([np.ones((X_test_scaled.shape[0], 1)), X_test_scaled])

Treinando:

In [ ]:
w = custom_ridge_regression(X_train_adj, y_train, lamb=0.1)

In [ ]:
y_pred1 = X_test_adj @ w

y_pred1 = pd.Series(y_pred1, index=y_test.index)

Comparando o y_pred1 com o y_test

In [ ]:
print(y_pred1)

513     5.957673
4177    6.768796
1308    5.214709
914     6.174433
3452    5.750230
          ...   
3772    5.635479
5191    5.968951
5226    5.398492
5390    5.499369
860     5.056690
Length: 1300, dtype: float64


In [ ]:
print(y_test)

513     7
4177    7
1308    5
914     6
3452    5
       ..
3772    5
5191    7
5226    5
5390    5
860     5
Name: quality, Length: 1300, dtype: int64


Calculando o RMSE:

In [ ]:
rmse_custom = np.sqrt(np.mean((y_test.values - y_pred1)**2))
print(rmse_custom)

0.737943999705386


Calculando o $R^2$

In [ ]:
ss_res = np.sum((y_test - y_pred1)**2)
ss_tot = np.sum((y_test - np.mean(y_test))**2)

r2_custom = 1 - ss_res / ss_tot
print("R² custom:", r2_custom)

R² custom: 0.28374181955336153


#### Ridge da Biblioteca

In [ ]:
model_rid = Ridge(alpha=0.1)   # Atenção!!! Nesse caso o lambda = alpha
model_rid.fit(X_train_scaled, y_train)

Ridge(alpha=0.1)

In [ ]:
y_pred2 = model_rid.predict(X_test_scaled)


RMSE da biblioteca

In [ ]:
rmse_lib = np.sqrt(np.mean((y_test.values - y_pred2)**2))
print(rmse_lib)

0.7379483417769509


$R^2$ da biblioteca

In [ ]:
ss_res = np.sum((y_test - y_pred2)**2)
ss_tot = np.sum((y_test - np.mean(y_test))**2)

r2_lib = 1 - ss_res / ss_tot
print("R² custom:", r2_custom)

R² custom: 0.28374181955336153


#### Diferenças entre os dois RIDGEs

Diferença entre os RMSEs

In [ ]:
diff_abs = abs(rmse_custom - rmse_lib)
print("Diferença absoluta:", diff_abs)

diff_percent = abs(rmse_custom - rmse_lib) / rmse_lib * 100
print(f"Diferença percentual: {diff_percent:.4f}%")

ratio = rmse_custom / rmse_lib
print("Razão (custom / lib):", ratio)

Diferença absoluta: 4.342071564855665e-06
Diferença percentual: 0.0006%
Razão (custom / lib): 0.9999941160223297


Diferença entre os $R^2$

In [ ]:
diff_abs = abs(r2_custom - r2_lib)
print("Diferença absoluta:", diff_abs)

diff_percent = abs(r2_custom - r2_lib) / r2_lib * 100
print(f"Diferença percentual: {diff_percent:.4f}%")

ratio = r2_custom / r2_lib
print("Razão (custom / lib):", ratio)

Diferença absoluta: 8.428968673590553e-06
Diferença percentual: 0.0030%
Razão (custom / lib): 1.0000297073554023


### K-Fold

In [ ]:
def best_k_fold(X, y, lambdas, k):
    n = len(y)
    indices = np.arange(n)
    np.random.shuffle(indices)

    folds = np.array_split(indices, k)

    rmse_per_lambda = {}

    for lamb in lambdas:
        rmse_list = []

        for i in range(k):
            val_idx = folds[i]
            train_idx = np.hstack([folds[j] for j in range(k) if j != i])

            # INDEXAÇÃO CORRETA
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val,   y_val   = X.iloc[val_idx],  y.iloc[val_idx]

            # modelo ridge feito do zero
            w = custom_ridge_regression(X_train.values, y_train.values, lamb)

            # predição
            y_pred = X_val.values @ w

            # RMSE
            rmse_list.append(np.sqrt(np.mean((y_val.values - y_pred)**2)))

        rmse_per_lambda[lamb] = np.mean(rmse_list)

    best_lambda = min(rmse_per_lambda, key=rmse_per_lambda.get)

    return best_lambda, rmse_per_lambda


Usando o 10-Fold

In [ ]:
lambdas = np.logspace(-4, 3, 10)  # 10 valores entre 1e-4 e 1e3

best_lambda, scores = best_k_fold(X_train, y_train, lambdas, k=10)

print("Melhor λ:", best_lambda)
print("RMSE médios:", scores)

Melhor λ: 0.1291549665014884
RMSE médios: {np.float64(0.0001): np.float64(0.7374738619096879), np.float64(0.0005994842503189409): np.float64(0.7374738170240362), np.float64(0.003593813663804626): np.float64(0.7374735500672382), np.float64(0.021544346900318846): np.float64(0.7374720253189828), np.float64(0.1291549665014884): np.float64(0.7374654395778042), np.float64(0.774263682681127): np.float64(0.7374911614061702), np.float64(4.641588833612782): np.float64(0.7381505096789273), np.float64(27.825594022071257): np.float64(0.7411936802014765), np.float64(166.81005372000593): np.float64(0.7534623736168701), np.float64(1000.0): np.float64(0.7689887492383182)}


#### Testando com o Lambda do 10-Fold implementado

In [ ]:
w1 = custom_ridge_regression(X_train_adj, y_train, best_lambda)

In [ ]:
y_pred3 = X_test_adj @ w1

y_pred3 = pd.Series(y_pred3, index=y_test.index)

Comparando o y_pred3 com o y_test

In [ ]:
print(y_pred3)

513     5.957647
4177    6.768752
1308    5.214678
914     6.174402
3452    5.750201
          ...   
3772    5.635443
5191    5.968916
5226    5.398457
5390    5.499342
860     5.056663
Length: 1300, dtype: float64


In [ ]:
print(y_test)

513     7
4177    7
1308    5
914     6
3452    5
       ..
3772    5
5191    7
5226    5
5390    5
860     5
Name: quality, Length: 1300, dtype: int64


Calculando o RMSE

In [ ]:
rmse_custom2 = np.sqrt(np.mean((y_test.values - y_pred3)**2))
print(rmse_custom2)

0.7379425591542906


Antigo RMSE -> Lambda = 1.0

In [ ]:
print(rmse_custom)

0.737943999705386


Diferença entre o RMSE calculado com lambda = 1.0 com o RMSE com Lambda Ideal

In [ ]:
diff_abs = abs(rmse_custom2 - rmse_custom)
print("Diferença absoluta:", diff_abs)

diff_percent = abs(rmse_custom2 - rmse_custom) / rmse_custom * 100
print(f"Diferença percentual: {diff_percent:.4f}%")

ratio = rmse_custom2 / rmse_custom
print("Razão (custom / lib):", ratio)

Diferença absoluta: 1.440551095410747e-06
Diferença percentual: 0.0002%
Razão (custom / lib): 0.9999980478856173


Calculando $R^2$

In [ ]:
ss_res1 = np.sum((y_test - y_pred3)**2)
ss_tot1 = np.sum((y_test - np.mean(y_test))**2)

r2_custom1 = 1 - ss_res1 / ss_tot
print("R² custom:", r2_custom1)

R² custom: 0.2837446159864233


Diferença entre os $R^2$

In [ ]:
diff_abs = abs(r2_custom1 - r2_custom)
print("Diferença absoluta:", diff_abs)

diff_percent = abs(r2_custom1 - r2_custom) / r2_custom * 100
print(f"Diferença percentual: {diff_percent:.4f}%")

ratio = r2_custom1 / r2_custom
print("Razão (custom / lib):", ratio)

Diferença absoluta: 2.7964330617802347e-06
Diferença percentual: 0.0010%
Razão (custom / lib): 1.0000098555548356


#### Testando com o Lambda do 10-Fold da biblioteca

In [ ]:
lambdas = np.logspace(-4, 3, 10)
scores = [cross_val_score(Ridge(alpha=l), X_train, y_train,
                          scoring="neg_root_mean_squared_error", cv=10).mean() for l in lambdas]

best_lambda = lambdas[np.argmax(scores)]
print("Melhor λ:", best_lambda)

Melhor λ: 0.0001


In [ ]:
w2 = custom_ridge_regression(X_train_adj, y_train, best_lambda)

In [ ]:
y_pred4 = X_test_adj @ w2

y_pred4 = pd.Series(y_pred4, index=y_test.index)

Calculando o RMSE

In [ ]:
rmse_custom3 = np.sqrt(np.mean((y_test.values - y_pred4)**2))
print(rmse_custom3)

0.7379489481179358


Calculando o $R^2$

In [ ]:
ss_res2 = np.sum((y_test - y_pred4)**2)
ss_tot2 = np.sum((y_test - np.mean(y_test))**2)

r2_custom2 = 1 - ss_res2/ ss_tot2
print("R² custom:", r2_custom2)

R² custom: 0.28373221353216194


#### Comparando o 10-fold Implementado com o da biblioteca

RMSE

In [ ]:
diff_abs = abs(rmse_custom3 - rmse_custom2)
print("Diferença absoluta:", diff_abs)

diff_percent = abs(rmse_custom3 - rmse_custom2) / rmse_custom2 * 100
print(f"Diferença percentual: {diff_percent:.4f}%")

ratio = rmse_custom3 / rmse_custom2
print("Razão (custom / lib):", ratio)

Diferença absoluta: 6.388963645131973e-06
Diferença percentual: 0.0009%
Razão (custom / lib): 1.0000086578061747


$R^2$

In [ ]:
diff_abs = abs(r2_custom2 - r2_custom1)
print("Diferença absoluta:", diff_abs)

diff_percent = abs(r2_custom2 - r2_custom1) / r2_custom1 * 100
print(f"Diferença percentual: {diff_percent:.4f}%")

ratio = r2_custom2 / r2_custom1
print("Razão (custom / lib):", ratio)

Diferença absoluta: 1.2402454261373208e-05
Diferença percentual: 0.0044%
Razão (custom / lib): 0.9999562900807183
